In [9]:
import pandas as pd
import datetime
from functools import reduce

In [2]:
def get_trade_time():
    """ 
    Return trade datettime
    """
    morning_trade_start_datetime = '09:30:00'
    morning_trade_end_datetime = '11:30:00'
    afternoon_trade_start_datetime = '13:00:00'
    afternoon_trade_end_datetime = '15:30:00'
    
    return (morning_trade_start_datetime, morning_trade_end_datetime, afternoon_trade_start_datetime,
    afternoon_trade_end_datetime)

In [3]:
def raw2tick(df):
    """
    处理原始数据：  
    - 同1秒只出现一次的数据，当成第0.5s数据处理
    - 交易时间过滤
    """

    # `Time`与下一个tick数据不同，则为第0.5s数据
    # 同1秒有两条数据时 -> 第二条数据
    # 同1秒只有一条数据 
    df['datetime'] = pd.to_datetime(df.Time)
    latter_indices = (df.Time != df.Time.shift(-1)) & df.datetime.map(lambda x: x.microsecond == 0)
    df.loc[latter_indices, 'datetime'] += datetime.timedelta(seconds=0.5)

    t = pd.to_timedelta(get_trade_time())
    ranges = list(zip(t[0::2], t[1::2]))
    def in_trade_time(x):
        x= datetime.timedelta(hours=x.hour, minutes=x.minute, seconds=x.second, microseconds=x.microsecond)
        return any(map(lambda r: r[0] <= x <= r[1], ranges))

    df = df[df.datetime.map(in_trade_time)]
    df.index = df.datetime

    return df

In [4]:
def tick2min(df):
    resampler = df.resample('1min', closed='right', label='left')
    df = resampler.LastClose.ohlc()
    return df

In [5]:
def raw2min(df):
    df = raw2tick(df)
    df = tick2min(df)
    return df

In [6]:
t = pd.to_timedelta(get_trade_time())
print(t)
ranges = list(zip(t[0::2], t[1::2]))
print(ranges)

TimedeltaIndex(['09:30:00', '11:30:00', '13:00:00', '15:30:00'], dtype='timedelta64[ns]', freq=None)
[(Timedelta('0 days 09:30:00'), Timedelta('0 days 11:30:00')), (Timedelta('0 days 13:00:00'), Timedelta('0 days 15:30:00'))]
